# Parser tests

In [17]:
[global]
from dsc.dsc_parser import DSC_Script as ds
from dsc.utils import FormatError

def test_format(obj, *args):
    try:
        obj(*args)
    except Exception as e:
        assert type(e).__name__ == 'FormatError'
        print(e)

## Basic syntax

In [18]:
%run
[1]
# missing output
text = '''
simulate: sim.R
    x: 1
    y: 2
'''
test_format(ds, text)

Invalid syntax ``s``. Should be in the format of ``module names: module executables``
